![Image](actinia_logo.png)

## Introduction

Actinia is a REST service to process geographical data that can be
managed by the GRASS GIS software system. The software is designed to
expose a GRASS GIS database and many [GRASS GIS](https://grass.osgeo.org/) processing tools as a
[REST service](https://en.wikipedia.org/wiki/Representational_State_Transfer).
Hence, access to GRASS resources like raster maps,
space-time raster datasets, processing and analysis modules are
available via URL. In addition, actinia allows the cloud based processing
of data, for example all Landsat 4-8 scenes as well as all
Sentinel-2 scenes in an ephemeral database. The computational results
of ephemeral processing are available via object storage as GeoTIFF/COG
or GeoPackage files.

The actinia service consists of the *[actinia core](https://github.com/actinia-org/actinia-core)*
that provides the basic but sophisticated processing service and
*[actinia plugins](https://github.com/orgs/mundialis/repositories?q=actinia+plugins&type=all&language=&sort=)*
that provide problem specific services like NDVI computation from Sentinel-2
or Landsat data, spatio-temporal statistical analysis and many more.

The following example is a Jupyter Notebook which uses the 
[actinia-python-client](https://actinia-org.github.io/actinia-python-client/) ([source code](https://github.com/actinia-org/actinia-python-client)) to
establish the connection to the actinia instance. 

## Installation of the actinia-python-client

(documentation: https://actinia-org.github.io/actinia-python-client/)

First we install the actinia-python-client.

In [ ]:
# install actinia-python-client
# for latest version, see https://github.com/actinia-org/actinia-python-client/releases
!wget -c https://github.com/actinia-org/actinia-python-client/releases/download/0.3.0/actinia_python_client-0.3.0-py3-none-any.whl

!pip3 install actinia_python_client-0.3.0-py3-none-any.whl

## Helper function for printing

Next we implement a helper function for "pretty printing" of actinia results:

In [ ]:
from json import dumps as json_dumps


def print_dict(input_dict, text=None):
    if text:
        print(text)
    if "region" in input_dict:
        input_dict["region"] = input_dict["region"].__dict__
    print(json_dumps(input_dict, sort_keys=True, indent=4))


def print_dict_keys(input_dict, text=None):
    if text:
        print(text)
    print(", ".join(input_dict.keys()))

## Connecting to the actinia instance using the actinia-python-client

Now we connect this session to the default actinia server which is defined in the actinia-python-client, currently https://actinia.mundialis.de.

In [ ]:
# connect to default actinia server (actinia.mundialis.de)
from actinia import Actinia

actinia_mundialis = Actinia()

# retrieve metadata about actinia and related software versions
version = actinia_mundialis.get_version()
print_dict(version, "Version is:")


Subsequently, we set the authentication settings of the actinia demo user to gain access to the
actinia server functionality. The user and password have exist on the server.

In [ ]:
# define user/password for connection
actinia_mundialis.set_authentication("demouser", "gu3st!pa55w0rd")
print("Connected to actinia server.")

## Location Management

With the location management the locations can be requested as well as
information of each location. Also a location can be created and deleted if the user is permitted.

Above, we have already connect the Jupyter notebook using the actinia Python client to
[actinia](https://actinia.mundialis.de/) and set the authentication.

Attention: The demouser is not permitted to create or delete a location!

### Retrieve the list of available locations and information about a selected location

The first task is to obtain the list of locations and retrieve the metadata of a selected location.

In [ ]:
# obtain the list of locations (which are accessible to current user)
locations = actinia_mundialis.get_locations()
print_dict_keys(locations, "Locations: ")

Retrieve the metadata of a selected location (note that there are two ways).

In [ ]:
# way 1, using the definition from above
print_dict(locations["nc_spm_08"].get_info(), "Location info (way 1):")

In [ ]:
# way 2, also specifying the server
print_dict(actinia_mundialis.locations["nc_spm_08"].get_info(), "Location info (way 2):")

### Creation of a new location

NOTE: `new location feature not available to the "demouser"`!

In [ ]:
# Create a new location
new_location = actinia_mundialis.create_location("test_location", 25832)
print(new_location.name)
print(new_location.region)
print_dict_keys(actinia_mundialis.locations)

### Deletion of a new location

NOTE: delete location not available to the "demouser".

In [ ]:
## Delete a location
actinia_mundialis.locations["test_location"].delete()
print_dict_keys(actinia_mundialis.locations)

## Mapset Management

With the mapset management the mapsets of a specified location can be
requested as well as information of each mapset.
Also a mapset can be created and deleted if the user is permitted.

In [ ]:
# request all locations
locations = actinia_mundialis.get_locations()

### Get mapsets of selected location
Get mapsets of the ***nc_spm_08*** location:

In [ ]:
mapsets = actinia_mundialis.locations["nc_spm_08"].get_mapsets()
print_dict_keys(mapsets, "Mapsets in nc_spm_08:")

### Create a mapset

In [ ]:
mapset_name = "test_mapset"
locations["nc_spm_08"].create_mapset(mapset_name)
print_dict_keys(mapsets, "Mapsets in nc_spm_08:")

### Delete a mapset

In [ ]:
locations["nc_spm_08"].delete_mapset(mapset_name)
print_dict_keys(mapsets, "Mapsets in nc_spm_08:")

## Raster and Vector Management

First request the list of all locations and then show the mapsets in a specific location.

In [ ]:
# request all locations
locations = actinia_mundialis.get_locations()
# Get Mapsets of nc_spm_08 location
mapsets = actinia_mundialis.locations["nc_spm_08"].get_mapsets()
print_dict_keys(mapsets, "Mapsets in nc_spm_08:")

## Raster manangement

Get all raster layers of the `PERMANENT` mapsets.

In [ ]:
rasters = mapsets["PERMANENT"].get_raster_layers()
print_dict_keys(rasters, "Raster maps:")

Get information of the raster map `zipcodes`.

In [ ]:
info = rasters["zipcodes"].get_info()
print_dict(info, "Zipcodes raster info:")

Upload a GeoTIFF file as raster layer to a user mapset (we first create the user mapset).

In [ ]:
mapset_name = "test_mapset"

# create mapset
locations["nc_spm_08"].create_mapset(mapset_name)

# upload tif
raster_layer_name = "test"
file = "/home/testuser/data/elevation.tif"
locations["nc_spm_08"].mapsets[mapset_name].upload_raster(raster_layer_name, file)
print_dict_keys(locations["nc_spm_08"].mapsets[mapset_name].raster_layers, "Raster maps in new mapset:")

Delete a raster layer.

In [ ]:
locations["nc_spm_08"].mapsets[mapset_name].delete_raster(raster_layer_name)
print_dict_keys(locations["nc_spm_08"].mapsets[mapset_name].raster_layers, "Raster maps in new mapset:")

# delete mapset
locations["nc_spm_08"].delete_mapset(mapset_name)

### Vector management

Get all vector maps in the `PERMANENT` mapset.

In [ ]:
vectors = mapsets["PERMANENT"].get_vector_layers()
print_dict_keys(vectors, "Vector maps:")

Get information of the vector map `boundary_county`.

In [ ]:
info = vectors["boundary_county"].get_info()

Upload a GeoJSON file as a vector layer to a user mapset (we first create the user mapset).

In [ ]:
# create mapset
mapset_name = "test_mapset"
locations["nc_spm_08"].create_mapset(mapset_name)

# upload tif
vector_layer_name = "test"
file = "/home/testuser/data/firestations.geojson"
locations["nc_spm_08"].mapsets[mapset_name].upload_vector(vector_layer_name, file)
print_dict_keys(locations["nc_spm_08"].mapsets[mapset_name].vector_layers, "Vectors in new mapset:")

Delete a vector layer.

In [ ]:
locations["nc_spm_08"].mapsets[mapset_name].delete_vector(vector_layer_name)
print_dict_keys(locations["nc_spm_08"].mapsets[mapset_name].vector_layers, "Vectors in new mapset:")

# delete mapset
locations["nc_spm_08"].delete_mapset(mapset_name)

## Process Chain Validation

A process chain can be validated before a job is started.

In [ ]:
# request all locations
locations = actinia_mundialis.get_locations()
print_dict_keys(locations, "Locations: ")

### Synchronous process chain validation

Why validation? It may happen that your JSON file to be sent to the endpoint contains a typo or other invalid content. For the identification of problems prior to executing the commands contained in the JSON file (which may last for hours), it is recommended to validate this file. For this, actinia can be used as it provides a validation endpoint.

In case of synchronous process chain validation we will wait until the validation job has finished.

In [ ]:
pc = {
    "list": [
      {
          "id": "r_mapcalc",
          "module": "r.mapcalc",
          "inputs": [
              {
                  "param": "expression",
                  "value": "baum=42"
              }
          ]
      },
      {
          "id": "exporter_1",
          "module": "exporter",
          "outputs": [
              {
                  "export": {"type": "raster", "format": "COG"},
                  "param": "map",
                  "value": "baum"
              }
          ]
      }
    ],
    "version": "1"
}

actinia_mundialis.locations["nc_spm_08"].validate_process_chain_sync(pc)

### Asynchronous process chain validation

In case of asynchronous process chain validation we have to poll (repeatedly check) until the validation job has finished.

In [ ]:
pc = {
    "list": [
      {
          "id": "r_mapcalc",
          "module": "r.mapcalc",
          "inputs": [
              {
                  "param": "expression",
                  "value": "baum=42"
              }
          ]
      },
      {
          "id": "exporter_1",
          "module": "exporter",
          "outputs": [
              {
                  "export": {"type": "raster", "format": "COG"},
                  "param": "map",
                  "value": "baum"
              }
          ]
      }
    ],
    "version": "1"
}

val_job = actinia_mundialis.locations["nc_spm_08"].validate_process_chain_async(pc, "r.mapcalc")
val_job.poll_until_finished()
print(val_job.status)
print(val_job.message)

## Processing

We recommend to start a processing job with a valid process chain (see above).

For running a processing job, first connect the Jupyter session through the actinia Python client with the [actinia](https://actinia.mundialis.de/) server and set the authentication properly.

In [ ]:
from actinia import Actinia

actinia_mundialis = Actinia()
actinia_mundialis.get_version()
actinia_mundialis.set_authentication("demouser", "gu3st!pa55w0rd")

# request all locations
locations = actinia_mundialis.get_locations()

### Ephemeral Processing

**Ephemeral processing** is used to keep computed results, including user-generated data and temporary data, only for a limited period of time (e.g. 24 hours by default in the actinia demo server). This reduces cloud storage costs.

In contrast, **persistent processing** refers to the persistent retention of data without a scheduled deletion time, even in the event of a power outage, resulting in corresponding storage costs. In the geo/EO context, persistent storage is used to provide, for example, basic cartography, i.e. elevation models, road networks, building footprints, etc.

Here an example for an ephemeral processing job: We use [r.relief](https://grass.osgeo.org/grass-stable/manuals/r.relief.html) to generate a hillshading map and pre-define the resolution to 10 m. The computational region is set to the input elevation map. The resulting `hillshade.tif` raster map is then provided as a resource for download.

In [ ]:
pc = {
    "list": [
        {
             "id": "computational_region",
             "module": "g.region",
             "inputs": [
                 {"param": "raster",
                  "value": "elevation@PERMANENT"},
                 {"param": "res",
                  "value": "10"}
             ],
             "stdout": {"id": "region", "format": "kv", "delimiter": "="},
             "flags": "g"
         },
        {
          "id": "create_hillshading",
          "module": "r.relief",
          "inputs": [
              {
                  "param": "input",
                  "value": "elevation"
              }
          ],
          "outputs": [
              {
                  "param": "output",
                  "value": "hillshade"
              }
          ]
      },
      {
          "id": "exporter_1",
          "module": "exporter",
          "outputs": [
              {
                  "export": {"type": "raster", "format": "COG"},
                  "param": "map",
                  "value": "hillshade"
              }
          ]
      }
    ],
    "version": "1"
}
job = actinia_mundialis.locations["nc_spm_08"].create_processing_export_job(pc, "hillshading")
job.poll_until_finished()

print(job.status)
print(job.message)
exported_raster = job.urls["resources"][0]
print(exported_raster)

The computed `hillshade.tif` output map should look as follows:

<center>
<img src="img/nc_hillshade.png" width="50%">
Fig: Hillshade map computed from elevation map.
</center>